<a href="https://colab.research.google.com/github/ch23s020/assignment2/blob/main/Assignment2_CH23S020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
from zipfile import ZipFile

# Mount Google Drive (authentication required only on first run)
drive.mount('/content/drive')

# Specify paths
zip_path = "/content/drive/MyDrive/nature_12K.zip"
target_dir = "/content/drive/MyDrive/Assignment_2"  # Note the double underscore

try:
  # Create the target directory (if it doesn't exist)
  !mkdir -p {target_dir}

  # Extract the zip file
  with ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(target_dir)

  print("Unzipped successfully!")

except Exception as e:
  print(f"Error unzipping: {e}")

# Unmount Google Drive (optional)
#drive.unmount('/content/drive')

In [2]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.1/266.1 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.0 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
import wandb
import numpy as np
import pandas as pd


In [ ]:

# Wandb Sweep and Hyperparameter values

sweep_config = {

    'method': 'random',

    'parameters': {

        'num_filters': {'values': [[16, 32], [32, 64], [64, 128]]},

        'filter_size': {'values': [[3, 3], [5, 5], [7, 7]]},

        'activation': {'values': ['ReLU', 'GELU', 'SiLU']},

        'use_batchnorm': {'values': [0, 1]},

        'use_dropout': {'values': [0, 1]},

        'lr': {'values': [0.001, 0.01, 0.1]},

        'num_epochs': {'values': [5, 10, 15]},

        'filter_org': {'values': ['same', 'different']},

        'data_augmentation': {'values': ['yes', 'no']},

        'batch_size': {'values': [32, 64, 128]},

        'num_neurons': {'values': [64, 128, 256]},

        'learning_algorithm': {'values': ['Adam', 'SGD']},

        'project': {'values': ['categorized', 'uncategorized']}

    }
}


sweep_id = wandb.sweep(sweep_config, project="Assignment2_CNN")

#training function

def train_sweep():

    # Initialize Wandb with project name
    wandb.init()

    # Specify the sweep configuration
    config_default = wandb.config

    # Start training with the current configuration

    for _ in range(config_default['num_epochs']):
        train(config_default)

    # Finish the Wandb run after completing all epochs to avoid broken pipe error

    wandb.finish()

def train(config):

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Data augmentation and normalization Step

    if config['data_augmentation'] == 'yes':

        train_transforms = transforms.Compose([
            transforms.RandomResizedCrop(224),

            transforms.RandomHorizontalFlip(),

            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),

            transforms.ToTensor(),

            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    else:
        train_transforms = transforms.Compose([
            transforms.Resize(256),

            transforms.CenterCrop(224),

            transforms.ToTensor(),

            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    test_transforms = transforms.Compose([

        transforms.Resize(256),

        transforms.CenterCrop(224),

        transforms.ToTensor(),

        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    # Data Loading and Pre-Processing Step (Using the data directly from google drive)
    train_data = datasets.ImageFolder(root='/content/drive/MyDrive/inaturalist_12K/train', transform=train_transforms)

    test_data = datasets.ImageFolder(root='/content/drive/MyDrive/inaturalist_12K/val', transform=test_transforms)

    # Split train_data into train and validation

    train_size = int(0.8 * len(train_data))

    val_size = len(train_data) - train_size

    train_dataset, val_dataset = random_split(train_data, [train_size, val_size])

    # DataLoader with multiprocessing adding num_worker = 2

    train_loader = DataLoader(dataset=train_dataset, batch_size=config['batch_size'], shuffle=True, num_workers=2)

    val_loader = DataLoader(dataset=val_dataset, batch_size=config['batch_size'], shuffle=False, num_workers=2)

    test_loader = DataLoader(dataset=test_data, batch_size=config['batch_size'], shuffle=False, num_workers=2)


#Step3: Class and Training Loop

    class CNN(nn.Module):

        def __init__(self, num_classes, num_filters, filter_size, activation, filter_org, use_batchnorm, use_dropout, num_neurons):

            super(CNN, self).__init__()

            self.num_classes = num_classes

            self.num_filters = num_filters

            self.filter_size = filter_size

            self.activation = activation

            self.filter_org = filter_org

            self.use_batchnorm = use_batchnorm

            self.use_dropout = use_dropout

            self.num_neurons = num_neurons

            # Convolution layers

            layers = []

            in_channels = 3 # Assuming RGB images

            prev_output_size = 224  # Initial image size, adjust as necessary

            for i in range(len(num_filters)):

                layers.append(nn.Conv2d(in_channels, num_filters[i], filter_size[i]))

                if use_batchnorm:

                    layers.append(nn.BatchNorm2d(num_filters[i]))

                if activation == 'ReLU':

                    layers.append(nn.ReLU())

                elif activation == 'GELU':

                    layers.append(nn.GELU())

                elif activation == 'SiLU':

                    layers.append(nn.SiLU())

                elif activation == 'Mish':

                    layers.append(nn.Mish())
                layers.append(nn.MaxPool2d(2, 2))

                if use_dropout:

                    layers.append(nn.Dropout(0.2))

                in_channels = num_filters[i]

                # Calculate the output size of this layer
                prev_output_size = (prev_output_size - filter_size[i] + 1) // 2

            self.conv_layers = nn.Sequential(*layers)

            # Calculate the input size to the fully connected layers
            self.fc_input_size = in_channels * prev_output_size * prev_output_size

            # Dense layers
            self.fc = nn.Linear(self.fc_input_size, num_neurons)

            self.fc2 = nn.Linear(num_neurons, num_classes)

        def forward(self, x):

            x = self.conv_layers(x)

            x = x.view(x.size(0), -1)

            x = self.fc(x)

            x = self.fc2(x)

            return x

    # Initialize the model

    model = CNN(num_classes=10, num_filters=config['num_filters'], filter_size=config['filter_size'],

                activation=config['activation'], filter_org=config['filter_org'],

                use_batchnorm=config['use_batchnorm'], use_dropout=config['use_dropout'],

                num_neurons=config['num_neurons']).to(device)

    # Loss function and optimizer

    criterion = nn.CrossEntropyLoss()

    if config['learning_algorithm'] == 'Adam':

        optimizer = optim.Adam(model.parameters(), lr=config['lr'])

    elif config['learning_algorithm'] == 'SGD':

        optimizer = optim.SGD(model.parameters(), lr=config['lr'])

    # Training loop

    for epoch in range(config['num_epochs']):
       # Use num_epochs from config

        model.train()

        running_loss = 0.0

        correct_train = 0

        total_train = 0

        for batch_idx, (data, targets) in enumerate(train_loader):

            data, targets = data.to(device), targets.to(device)  # Move data to device

            # Forward_Prop

            outputs = model(data)

            loss = criterion(outputs, targets)

            # Backward_prop

            optimizer.zero_grad()

            loss.backward()

            optimizer.step()

            running_loss += loss.item()

            # Calculating training accuracy

            _, predicted = torch.max(outputs.data, 1)

            total_train += targets.size(0)

            correct_train += (predicted == targets).sum().item()

            # Printing training accuracy after each batch

            train_accuracy = 100 * correct_train / total_train

            print(f'Epoch [{epoch+1}/{config["num_epochs"]}], Batch [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}, Train Acc: {train_accuracy:.2f}%')

        # Log training loss and accuracy

        train_loss = running_loss / len(train_loader)

        wandb.log({'train_loss': train_loss, 'train_accuracy': train_accuracy}, step=epoch)

        # Validation loop

        model.eval()

        with torch.no_grad():

            correct_val = 0

            total_val = 0

            val_loss = 0.0

            for data, targets in val_loader:

                data, targets = data.to(device), targets.to(device)
                  # Move data to device

                outputs = model(data)

                val_loss += criterion(outputs, targets).item()

                # Calculating validation accuracy
                _, predicted = torch.max(outputs.data, 1)

                total_val += targets.size(0)

                correct_val += (predicted == targets).sum().item()

            val_accuracy = 100 * correct_val / total_val

            print(f'Epoch [{epoch+1}/{config["num_epochs"]}], Val Loss: {val_loss/len(val_loader):.4f}, Val Acc: {val_accuracy:.2f}%')

            # Log validation loss
            wandb.log({'val_loss': val_loss/len(val_loader)}, step=epoch)

            # Log metrics to Wandb
            wandb.log({'val_accuracy': val_accuracy})

            # # Log parallel coordinate plot

            # num_samples = min(len(train_data), 1000)

            # indices = torch.randperm(len(train_data))[:num_samples]

            # train_subset = torch.utils.data.Subset(train_data, indices)

            # dataloader = torch.utils.data.DataLoader(train_subset, batch_size=num_samples)

            # images, labels = next(iter(dataloader))

            # images, labels = images.to(device), labels.to(device)

            # model.eval()

            # outputs = model(images)

            # _, preds = torch.max(outputs, 1)

            # df = pd.DataFrame({
            #     'Label': labels.cpu().numpy(),
            #     'Prediction': preds.cpu().numpy()
            # })
            # wandb.log({"parallel_coordinate_plot": wandb.plot.parallel_coordinates(df, 'Label')}, step=epoch)

            # # Log correlation plot
            # corr_matrix = np.corrcoef(labels.cpu().numpy(), preds.cpu().numpy())

            # wandb.log({"correlation_plot": wandb.plot.correlation_matrix(corr_matrix)}, step=epoch)

# Run the sweep
wandb.agent(sweep_id, function=train_sweep)
